In [29]:
# Dependencies and file read
import pandas as pd
import numpy as np
import os

file = os.path.join('purchase_data.json')

purchase_data = pd.read_json(file)

# View the data
purchase_data.head()

,Age,Gender,Item ID,Item Name,Price,SN
0,38,Male,165,Bone Crushing Silver Skewer,3.37,Aelalis34
1,21,Male,119,"Stormbringer, Dark Blade of Ending Misery",2.32,Eolo46
2,34,Male,174,Primitive Blade,2.46,Assastnya25
3,21,Male,92,Final Critic,1.36,Pheusrical25
4,23,Male,63,Stormfury Mace,1.27,Aela59


In [30]:
#Find player count by finding unique screen names and finding the length of that list
player_count = len(pur_data['SN'].unique())

# DataFrame creation for player count
players_df = pd.DataFrame([{'Total Players': player_count}])
#gets rid of number index and resets to Total Players 
players_df.set_index('Total Players', inplace = True)
players_df

""
Total Players
573


In [47]:
#Purchasing Analysis Total

# Creates a df but only keeping last occurance of Item ID
no_duplicate_items = purchase_data.drop_duplicates(['Item ID'], keep = 'last')

# Counts items by unique ID
total_unique = len(no_dup_items)

# Fnds the number of total purchases by counting occurances of price
total_purchase = purchase_data['Price'].count()

# Calculates total revenue for table by summing occurance of price and below calc
total_rev = round(purchase_data['Price'].sum(),2)

# Calculates total_rev
avg_price = round(total_rev / total_purchase, 2)

# Creates Purchase Analysis DataFrame
purchse_analysis = pd.DataFrame([{
    
    "Total Unique Items": total_unique,
    'Average Purchase Price': avg_price,
    'Total Purchases': total_purchase,
    'Total Revenue': total_rev
}])

# Print
purchse_analysis

,Average Purchase Price,Total Purchases,Total Revenue,Total Unique Items
0,2.93,780,2286.33,183


In [44]:
# Gender Information

# Creates df of unique player names by only keeping the last occurance
no_duplicate_players = purchase_data.drop_duplicates(['SN'], keep ='last')

# Counts gender values with no duplicate names
gender_counts = no_duplicate_players['Gender'].value_counts().reset_index()

# Adds column for % of players
gender_counts['% of Players'] = gender_counts['Gender'] / player_count * 100

# Renames columns
gender_counts.rename(columns = {'index': 'Gender', 'Gender': '# of Players'}, inplace = True)

# Sets index as Gender for aesthetics 
gender_counts.set_index(['Gender'], inplace = True)

# Print
gender_counts

,# of Players,% of Players
Gender,,
Male,465,81.151832
Female,100,17.452007
Other / Non-Disclosed,8,1.396161


In [43]:
# Purchasing Analysis by gender

# Counts purchases by gender
purchase_count_by_gender = pd.DataFrame(purchase_data.groupby('Gender')['Gender'].count())

# Sums price by gender
total_purchase_by_gender = pd.DataFrame(purchase_data.groupby('Gender')['Price'].sum())

# Merges the two data frames from above
purchase_analysis_gender = pd.merge(purchase_count_by_gender, total_purchase_by_gender, left_index = True, right_index = True)

# Renames columns
purchase_analysis_gender.rename(columns = {'Gender': '# of Purchases', 'Price':'Total Purchase Value'}, inplace=True)

# Adds column for average purchase price by gender by dividing total purcahse value by gender by # of purchases by gender
purchase_analysis_gender['Average Purchase Price'] = purchase_analysis_gender['Total Purchase Value']/purchase_analysis_gender['# of Purchases']

# Merges gender counts from above table (excluding dup SNs) into current df 
purchase_analysis_gender = purchase_analysis_gender.merge(gender_counts, left_index = True, right_index = True)

# Calculates and adds normalized total column by dividing total purchase value by unique # of players by genger
purchase_analysis_gender['Normalized Totals'] = purchase_analysis_gender['Total Purchase Value'] / purchase_analysis_gender['# of Players']
purchase_analysis_gender

# Deletes columns not needed for table (# of Players was used for normalized totals while % of players came from gender count table)
del purchase_analysis_gender['% of Players']
del purchase_analysis_gender['# of Players']

# Print
purchase_analysis_gender

,# of Purchases,Total Purchase Value,Average Purchase Price,Normalized Totals
Gender,,,,
Female,136,382.91,2.815515,3.829100
Male,633,1867.68,2.950521,4.016516
Other / Non-Disclosed,11,35.74,3.249091,4.467500


In [42]:
# Creates a column 'age_bin' based on conditional of age range
purchase_data.loc[(pur_data['Age'] < 10), 'age_bin'] = "< 10"
purchase_data.loc[(pur_data['Age'] >= 10) & (purchase_data['Age'] <= 14), 'age_bin'] = "10 - 14"
purchase_data.loc[(pur_data['Age'] >= 15) & (purchase_data['Age'] <= 19), 'age_bin'] = "15 - 19"
purchase_data.loc[(pur_data['Age'] >= 20) & (purchase_data['Age'] <= 24), 'age_bin'] = "20 - 24"
purchase_data.loc[(pur_data['Age'] >= 25) & (purchase_data['Age'] <= 29), 'age_bin'] = "25 - 29"
purchase_data.loc[(pur_data['Age'] >= 30) & (purchase_data['Age'] <= 34), 'age_bin'] = "30 - 34"
purchase_data.loc[(pur_data['Age'] >= 35) & (purchase_data['Age'] <= 39), 'age_bin'] = "35 - 39"
purchase_data.loc[(pur_data['Age'] >= 40), 'age_bin'] = "> 40"

# Counts purchases by age bin by counting screen names (non-unique)
purchase_count_age = pd.DataFrame(purchase_data.groupby('age_bin')['SN'].count())

# Finds avg price of purchases by age bin
avg_price_age = pd.DataFrame(purchase_data.groupby('age_bin')['Price'].mean())

# Finds total purchase value by age bin
total_purchase_age = pd.DataFrame(purchase_data.groupby('age_bin')['Price'].sum())

# Players by age bin
no_dup_age = pd.DataFrame(purchase_data.drop_duplicates('SN', keep = 'last').groupby('age_bin')['SN'].count())

# Merges all info from above into one df
merge_age = pd.merge(purchase_count_age, avg_price_age, left_index = True, right_index = True).merge(total_purchase_age, left_index = True, right_index = True).merge(no_dup_age, left_index = True, right_index = True)

# Renames columns
merge_age.rename(columns = {"SN_x": "# of Purchases", "Price_x": "Average Purchase Price", "Price_y": "Total Purchase Value", "SN_y": "# of Purchasers"}, inplace = True)

# Calculates normalized totals
merge_age['Normalized Totals'] = merge_age['Total Purchase Value']/merge_age['# of Purchasers']

# Rest index for aesthetics
merge_age.index.rename("Age", inplace = True)

# Print
merge_age

,# of Purchases,Average Purchase Price,Total Purchase Value,# of Purchasers,Normalized Totals
Age,,,,,
10 - 14,35,2.770000,96.95,23,4.215217
15 - 19,133,2.905414,386.42,100,3.864200
20 - 24,336,2.913006,978.77,259,3.779035
25 - 29,125,2.962640,370.33,87,4.256667
30 - 34,64,3.082031,197.25,47,4.196809
35 - 39,42,2.842857,119.40,27,4.422222
< 10,28,2.980714,83.46,19,4.392632
> 40,17,3.161765,53.75,11,4.886364


In [49]:
# Identify the the top 5 spenders in the game

# Group by screen name to find, total purchase per person, number of purchases per person, and average price price per person
purchase_amountt_by_SN = pd.DataFrame(purchase_data.groupby('SN')['Price'].sum())
num_purchase_by_SN = pd.DataFrame(purchase_data.groupby('SN')['Price'].count())
avg_purchase_by_SN = pd.DataFrame(purchase_data.groupby('SN')['Price'].mean())

# merge the  dfs
merged_top5 = pd.merge(purchase_amt_by_SN, num_purchase_by_SN, left_index = True, right_index = True).merge(avg_purchase_by_SN, left_index=True, right_index=True)

# rename columns
merged_top5.rename(columns = {'Price_x': 'Total Purchase Value', 'Price_y':'Purchase Count', 'Price':'Average Purchase Price'}, inplace = True)

# sort from highest purchase value to lowest
merged_top5.sort_values('Total Purchase Value', ascending = False, inplace=True)

# take top 5 only
merged_top5 = merged_top5.head()

# print
merged_top5

,Total Purchase Value,Purchase Count,Average Purchase Price
SN,,,
Undirrala66,17.06,5,3.412000
Saedue76,13.56,4,3.390000
Mindimnya67,12.74,4,3.185000
Haellysu29,12.73,3,4.243333
Eoda93,11.58,3,3.860000


In [51]:
# Identify the 5 most popular items
# Count each item
top5_items_ID = pd.DataFrame(purchase_data.groupby('Item ID')['Item ID'].count())

# Sort the pruchase counts
top5_items_ID.sort_values('Item ID', ascending = False, inplace = True)

# Find the total purchase for each item
top5_items_total = pd.DataFrame(purchase_data.groupby('Item ID')['Price'].sum())

# Merge
top5_items = pd.merge(top5_items_ID, top5_items_total, left_index = True, right_index = True)

# Drop duplicates
no_dup_items = purchase_data.drop_duplicates(['Item ID'], keep = 'last')

# Merge
top5_merge_ID = pd.merge(top5_items, no_dup_items, left_index = True, right_on = 'Item ID')

# Organize columns
top5_merge_ID = top5_merge_ID[['Item ID', 'Item Name', 'Item ID_x', 'Price_y', 'Price_x']]

# rename columns
top5_merge_ID.rename(columns =  {'Item ID_x': 'Purchase Count', 'Price_y': 'Item Price', 'Price_x': 'Total Purchase Value'}, inplace=True)

# Print
top5_merge_ID.head()

,Item ID,Item Name,Purchase Count,Item Price,Total Purchase Value
721,39,"Betrayal, Whisper of Grieving Widows",11,2.35,25.85
766,84,Arcane Gem,11,2.23,24.53
772,31,Trickster,9,2.07,18.63
761,175,Woeful Adamantite Claymore,9,1.24,11.16
765,13,Serenity,9,1.49,13.41


In [52]:
# Most Profitable Items
# Find total purchase values
top5_profit = pd.DataFrame(purchase_data.groupby('Item ID')['Price'].sum())
top5_profit.sort_values('Price', ascending = False, inplace = True)

# Top 5
top5_profit = top5_profit.iloc[0:5][:]

# Item Purchase count
purchase_count_profit = pd.DataFrame(purchase_data.groupby('Item ID')['Item ID'].count())

top5_profit = pd.merge(top5_profit, purchase_count_profit, left_index = True, right_index = True, how = 'left')
top5_merge_profit = pd.merge(top5_profit, no_dup_items, left_index = True, right_on = 'Item ID', how = 'left')
top5_merge_profit = top5_merge_profit[['Item ID', 'Item Name', 'Item ID_x', 'Price_y','Price_x']]
top5_merge_profit.set_index(['Item ID'], inplace=True)
top5_merge_profit.rename(columns = {'Item ID_x': 'Purchase Count', 'Price_y': 'Item Price', 'Price_x': 'Total Purchase Value'}, inplace = True)
top5_merge_profit.style.format({'Item Price': '${:.2f}', 'Total Purchase Value': '${:.2f}'})

,Item Name,Purchase Count,Item Price,Total Purchase Value
Item ID,,,,
34,Retribution Axe,9,$4.14,$37.26
115,Spectral Diamond Doomblade,7,$4.25,$29.75
32,Orenmir,6,$4.95,$29.70
103,Singed Scalpel,6,$4.87,$29.22
107,"Splitter, Foe Of Subtlety",8,$3.61,$28.88


In [53]:
# Highest priced items
highest_priced = no_dup_items.sort_values('Price', ascending = False)
highest_priced[['Item ID', 'Item Name', 'Price']].head()

,Item ID,Item Name,Price
657,32,Orenmir,4.95
670,177,"Winterthorn, Defender of Shifting Worlds",4.89
716,103,Singed Scalpel,4.87
336,173,Stormfury Longsword,4.83
419,42,The Decapitator,4.82


In [54]:
# Lowest priced items
lowest_priced = no_dup_items.sort_values('Price', ascending = True)
lowest_priced[['Item ID', 'Item Name', 'Price']].head()

,Item ID,Item Name,Price
667,15,Soul Infused Crystal,1.03
771,25,Hero Cane,1.03
624,95,Singed Onyx Warscythe,1.03
723,69,"Frenzy, Defender of the Harvest",1.06
430,74,Yearning Crusher,1.06


In [55]:
# Gender Purchases
percent_total_gen = pur_analysis_gen['Total Purchase Value'] / total_rev
percent_total_gen

Gender
Female                   0.167478
Male                     0.816890
Other / Non-Disclosed    0.015632
Name: Total Purchase Value, dtype: float64